In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import math

exps = (NoSuchElementException, StaleElementReferenceException)

In [2]:
def get_jobs(keyword, pages, verbose=False):
    
    """Method to collect job listings from Glassdoor.
        
        Args: 
            keyword (string): the name of the job title for which you want to collect jobs
            pages (int): the number of pages you want to collect(around 30 jobs/page)
        
        Returns: 
            pandas.dataframe: a dataframe with all the jobs and their details
    
        """
    
    options = webdriver.ChromeOptions()
    
    exec_path_mac = '/Users/mahmoudhamra/Dropbox/GitHub Projects/da_salary_proj/scraping-glassdoor-selenium-master/chromedriver'
    exec_path_pc = 'C:/Users/mahmo/Dropbox/GitHub Projects/da_salary_proj/chromedriver'
    driver = webdriver.Chrome(executable_path=exec_path_pc, options=options)
    

    driver.set_window_size(1120, 1000)
    url = 'https://www.glassdoor.com/Job/' + keyword.split()[0] + '-' + keyword.split()[1] + '-jobs-SRCH_KO0,12.htm?includeNoSalaryJobs=true&sortBy=date_desc'    
    driver.get(url)
    df = []    
    
    for i in range(pages):
        
        if i>=1:
            url = 'https://www.glassdoor.com/Job/data-analyst-jobs-SRCH_KO0,12_IP' + str(i+1) + '.htm?includeNoSalaryJobs=true&sortBy=date_desc'
            driver.get(url)
        else:
            pass
        
        driver.implicitly_wait(1)
        #collect job listings in the page (30 jobs/page)
        #jobs = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div[1]/div/div[2]/section/article/div[1]/ul/li')
        
        
        jobs = driver.find_elements_by_xpath('.//*[@id="MainCol"]/div[1]/ul/li')
        
        for job in jobs:
            
            print("---------------------------")
            
            # click on job 
            job.click()
            print("JOB BUTTON CLICKED")
            
            #if sign up window pops up close it
            try:
                driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
                print("clicked the X button")
            except NoSuchElementException:
                pass

            
            #get company name
            try:
                company_name = job.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/').text
            except exps:
                company_name = np.nan
                
            try:
                
                job_title = job.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[2]').text
            except exps:
                job_title = np.nan
                
            
            try:
                rating = job.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/span').text
            except exps:
                rating = np.nan
                
            try:
                salary_estimate = job.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span').text
            except exps:
                salary_estimate = np.nan
            
            try:
                location = job.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text
            except exps:
                location = np.nan
                                

            #Looping over Company Overview section and appending information to comp_info list
            comp_info = []
            elems = driver.find_elements_by_xpath('.//div[@id="EmpBasicInfo"]//div//div[@class="d-flex flex-wrap"]//div[@class="d-flex justify-content-start css-rmzuhb e1pvx6aw0"]')
            for element in elems:
                try:
                    driver.implicitly_wait(5)
                    #taking key value pairs ex: 'size', 1001 to 5000 Employees
                    comp_info.append(element.find_element_by_xpath('.//span[@class="css-1taruhi e1pvx6aw1"]').text)
                    comp_info.append(element.find_element_by_xpath('.//span[@class="css-i9gxme e1pvx6aw2"]').text)

                except StaleElementReferenceException:
                    print("##################StaleElementReferenceException")
                        
                    
            # Convert a list to dictionary
            it = iter(comp_info)
            comp_info = dict(zip(it, it))
            
           

            try:
                size = comp_info['Size']
            except KeyError:
                size = "Unknown"
                
                            
            try:
                founded = comp_info['Founded']
            except KeyError:
                founded = np.nan
                

            try:
                type_of_ownership = comp_info['Type']
            except KeyError:
                type_of_ownership = np.nan
                

            try:
                industry = comp_info['Industry']
            except KeyError:
                industry = np.nan
                

            try:
                sector = comp_info['Sector']
            except KeyError:
                sector = np.nan
                

            try:
                revenue = comp_info['Revenue']
            except KeyError:
                revenue = np.nan
                
            if verbose:
                print('job_title:' , job_title)
                print('salary_estimate:', salary_estimate)
                print('location:', location)
                print('rating:', rating)
                print('size:', size)
                print('founded:', founded)
                print('type_of_ownership:', type_of_ownership)
                print('industry:', industry)
                print('sector:', sector)
                print('revenue:', revenue)
                
                
            print("---------------------------")
            
            #add job details to df list
            df.append({
            "company_name" : company_name,
            "job_title": job_title,
            "salary_estimate": salary_estimate,
            "location": location,
            "rating" : rating,
            "size" : size,
            "founded" : founded,
            "type_of_ownership" : type_of_ownership,
            "industry" : industry,
            "sector" : sector,
            "revenue" : revenue})
         
    return pd.DataFrame(df)  #This line converts the dictionary object into a pandas DataFrame  
        
    

In [ ]:
df = get_jobs("data analyst", 30, verbose= True)

---------------------------
JOB BUTTON CLICKED
clicked the X button
job_title: Data Analyst Internship
salary_estimate: nan
location: Houston, TX
rating: 4.0
size: 10000+ Employees
founded: 1969
type_of_ownership: Company - Public
industry: Investment Banking & Asset Management
sector: Finance
revenue: $5 to $10 billion (USD)
---------------------------
---------------------------
JOB BUTTON CLICKED
job_title: Data Analyst
salary_estimate: nan
location: Boston, MA
rating: 4.4
size: 51 to 200 Employees
founded: 2007
type_of_ownership: Company - Private
industry: Staffing & Outsourcing
sector: Business Services
revenue: $25 to $50 million (USD)
---------------------------
---------------------------
JOB BUTTON CLICKED


In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
df.to_csv('jobs3', index=False)